In [1]:
from mand.core import Entity, node, getNode
from mand.core import ObjectDb, Timestamp, Context, _tr
from mand.core import ProfileMonitor, SummaryMonitor
from mand.core import find, addFootnote
from mand.demos.trading import makeWorld, bookSomeTrades, PnLExplainReport
from mand.lib.dbsetup import setUpDb

db = ObjectDb()
setUpDb(db)

In [2]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

p1.setChildren(p1.children() + [b2])
ts7 = Timestamp()

# Sanity check of a computation we might want to optimize [Test]

The PnL report is a good test, but at 152K getValues, it's a bit too big to immediately use:

In [4]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

r = PnLExplainReport(valuable=p, ts1=eod, ts2=ts7)

with ProfileMonitor():
    with SummaryMonitor():
        r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# Compute activity summary (19.32 seconds of wall clock time)

|key|value|
|-|-|
|Context|11
|Db.Get|2176
|GetValue|152158
|GetValue/Calc|2891


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|19,314,457|17|19,314,457|GetValue
|PnLExplainReport:data|1|19,314,440|453|19,314,440|GetValue/Calc
|TradingContainer:NPV|11|19,252,143|149|1,750,194|GetValue
|TradingContainer:NPV|11|19,251,994|94|1,750,181|GetValue/Calc
|Portfolio:items|121|19,089,115|1,332|157,761|GetValue
|Portfolio:items|121|19,089,011|922|157,760|GetValue/Calc
|Workbook:items|1,104|18,093,355|12,315|16,388|GetValue
|Workbook:items|1,100|18,081,040|9,427|16,437|GetValue/Calc
|PnLExplainReport:cutoffs|1|10,598,902|25|10,598,902|GetValue
|PnLExplainReport:cutoffs|1|10,598,877|195|10,598,877|GetValue/Calc
|Root:Clocks|548|10,598,592|1,969,183|19,340|Context
|TradeOpenEvent|1,014|4,272,808|4,272,808|4,213|Db.Get
|TradeOpenEvent:ticket|22,286|3,759,164|126,122|168|GetValue
|TradingTicket|1,014|3,633,042|3,633,042|3,582|Db.Get
|Root:Activity Portfolio|263|1,130,841|1,048,071|4,299|Context
|Root:Amend Trading|255|1,074,450|993,193|4,213|Context
|Root:Amend MarketData|255|1,050,181|970,469|4,118|Context
|Root:Start breaks|255|991,023|911,485|3,886|Context
|Portfolio:children|242|952,294|1,771|3,935|GetValue
|Portfolio:children|121|950,523|1,006|7,855|GetValue/Calc
|Root:End|278|934,644|853,899|3,362|Context
|Root:Activity Trading|263|893,194|813,660|3,396|Context
|Root:Start|270|891,946|811,326|3,303|Context
|Root:Activity MarketData|255|881,104|801,907|3,455|Context
|Root:Amend Portfolio|255|864,099|785,115|3,388|Context
|PortfolioUpdateEvent:children|121|398,031|3,041|3,289|GetValue
|TradingBook|102|365,144|365,144|3,579|Db.Get
|Equity:NPV|15|160,509|178|10,700|GetValue
|Equity:NPV|15|160,330|114|10,688|GetValue/Calc
|MarketInterface:spot|15|152,338|150|10,155|GetValue
|MarketInterface:spot|15|152,187|113|10,145|GetValue/Calc
|ExternalRefData:state|15|144,063|148|9,604|GetValue
|ExternalRefData:state|15|143,915|125|9,594|GetValue/Calc
|RefData:state|15|143,356|147|9,557|GetValue
|RefData:state|15|143,208|120|9,547|GetValue/Calc
|_WorkItemEvent:book2|22,286|123,085|115,859|5|GetValue
|_WorkItemEvent:book1|22,286|116,208|116,208|5|GetValue
|TradeOpenEvent:quantity|22,286|104,668|104,668|4|GetValue
|Clock:cutoffs|2,480|102,808|10,259|41|GetValue
|Clock:cutoffs|20|92,627|146|4,631|GetValue/Calc
|Clock:parent|20|91,689|192|4,584|GetValue
|Clock:parent|20|91,496|174|4,574|GetValue/Calc
|_WorkItemEvent:item|11,143|86,082|79,296|7|GetValue
|TradeOpenEvent:action|11,143|78,670|78,670|7|GetValue
|TradeOpenEvent:premium|11,143|75,086|71,809|6|GetValue
|TradeOpenEvent:unitPrice|11,143|63,995|63,995|5|GetValue
|Event:amends|11,340|61,556|61,556|5|GetValue
|TradingBook:clock|2,200|58,552|21,933|26|GetValue
|PortfolioUpdateEvent|12|54,964|54,964|4,580|Db.Get
|TradingPortfolio|10|37,071|37,071|3,707|Db.Get
|TradingBook:clock|1,100|36,619|12,216|33|GetValue/Calc
|RefDataUpdateEvent|9|36,586|36,586|4,065|Db.Get
|Clock|5|17,535|17,535|3,507|Db.Get
|Portfolio:clock|242|10,003|2,578|41|GetValue
|Portfolio:books|231|9,535|1,885|41|GetValue
|Portfolio:books|121|8,052|915|66|GetValue/Calc
|ClockEvent|2|7,978|7,978|3,989|Db.Get
|Portfolio:clock|121|7,425|1,331|61|GetValue/Calc
|MarketInterface:source|15|7,412|134|494|GetValue
|Equity:refdata|15|7,326|162|488|GetValue
|MarketInterface:source|15|7,278|101|485|GetValue/Calc
|Equity:refdata|15|7,163|115|477|GetValue/Calc
|Equity|2|6,785|6,785|3,392|Db.Get
|ClockEvent:parent|8|6,556|79|819|GetValue
|MarketInterface|2|6,459|6,459|3,229|Db.Get
|MarketDataSource|2|6,317|6,317|3,158|Db.Get
|MarketDataSource:clock|30|4,883|282|162|GetValue
|MarketDataSource:clock|15|4,600|153|306|GetValue/Calc
|RootClock|1|3,580|3,580|3,580|Db.Get
|ForwardCashflow|1|3,277|3,277|3,277|Db.Get
|Entity:clock|40|920|406|23|GetValue
|Entity:clock|20|514|208|25|GetValue/Calc
|ForwardCashflow:NPV|11|418|163|38|GetValue
|MarketInterface:sourceName|15|385|143|25|GetValue
|RefDataUpdateEvent:data|58|301|301|5|GetValue
|RootClock:cutoffs|53|274|207|5|GetValue
|ForwardCashflow:NPV|11|254|100|23|GetValue/Calc
|MarketInterface:sourceName|15|241|147|16|GetValue/Calc
|Equity:assetName|15|114|114|7|GetValue
|RootClock:cutoffs|1|66|35|66|GetValue/Calc
|RootClock:cosmicAll|1|18|9|18|GetValue
|CosmicAll:dbState|1|12|7|12|GetValue
|PnLExplainReport:valuable|2|12|12|6|GetValue
|RootClock:cosmicAll|1|9|9|9|GetValue/Calc
|PnLExplainReport:ts1|2|9|9|4|GetValue
|PnLExplainReport:ts2|2|8|8|4|GetValue
|CosmicAll:dbState|1|5|5|5|GetValue/Calc

# Caching/Reusing results

Until now, we have just checked to see if the current context contains a value for a node we are asking for, and if so, reuse that.

A better approach to caching bound *fn* on *object* in *context0* is:
    
1. Has fn been tweaked in context0? If so, return that
2. Is fn sufficiently trivial that we can avoid managing it as a node?
  * If trivial, just treat it as a pure python function and call it
  * Meta-data (inputs, outputs, etc) will be given to its caller and callees as appropriate
3. Ask object for context1
  * *context1* is a simplified version of *context0*
  * The default case is just to return *context1*
  * IRL, this could actually be a list of contexts or a pattern to match contexts against
4. Is fn cached in *context1*? If so, use that
5. Compute *fn* in *context1*
6. Construct *context2* from the inputs of the computed value
7. If *context2* is a subset of *context1*, cache *fn* in *context1*
8. Something odd happened
  * Footnote the problem as part of computation notes on the node's metadata
  * Maybe cache the node anyway in *context2*
    * Perhaps *object* will return *context2* as a siplification for future computations?
9. Return the value

Notes:
* Parallel compute of nodes not considered yet
* Context simplification (step 3) and input simplification (step 9) are probably intimately related
* The split between calculation and caching is nice:
  * BAs can write business logic
  * Computer scientists can add caching logic where needed
* We can use a profiler type object to gather runtime compute cost infomation
  * The resulting trace can be used as input to drive step 2 and step 3
    

In [5]:
dba = db.copy()
valuable = dba.get(b4.meta.path())

clock = valuable.getObj(_tr.RootClock, 'Main')
with SummaryMonitor():
    with Context({clock.cutoffs: ts7}):
        print valuable.NPV()
        node = getNode(valuable.NPV)
node.printInputGraph()

40.85


# Compute activity summary (0.18 seconds of wall clock time)

|key|value|
|-|-|
|Context|1
|Db.Get|35
|GetValue|185
|GetValue/Calc|24

<Node: /Global/TradingBook/Rates0.(TradingContainer:NPV)() in Root:4512165784 @4811707216 T=False>, nIn=3
   <Node: /Global/Equity/71475ae5-2823-4c52-b08c-7ef83264d743.(Equity:NPV)() in Root:4512165784 @4618148176 T=False>, nIn=2
      <Node: /Global/Equity/71475ae5-2823-4c52-b08c-7ef83264d743.(Equity:refdata)() in Root:4512165784 @4618148304 T=False>, nIn=1
         <Node: /Global/Equity/71475ae5-2823-4c52-b08c-7ef83264d743.(Equity:assetName)() in Root:4512165784 @4618148432 T=False>, nIn=0
      <Node: /Global/MarketInterface/GOOG.(MarketInterface:spot)() in Root:4512165784 @4618149200 T=True>, nIn=2
         <Node: /Global/MarketDataSource/source1.GOOG.(ExternalRefData:state)() in Root:4512165784 @4641092368 T=False>, nIn=1
            <Node: /Global/MarketDataSource/source1.GOOG.(RefData:state)() in Root:4512165784 @4641092432 T=False>, nIn=10
               <Node: /Global/Clock/MarketData.(Clock:cutoffs)() in Root:4512165784 @4663107984 T=True>, nIn=2
                  <Node: /Glo

# Graph Simplification

* Very simple input management simplifier: ignore inputs if they are leaves and not tweakable
* Simple context simplification: just switch to a simple context for certain known methods

In [6]:
from mand.graph import DependencyManager, setDependencyManager
from mand.core import Event

class DM1(DependencyManager):
    
    def __init__(self):
        self.contexts = {}
        self.simpleClockMethods = ('Workbook:items', 'RefData:state', 'Portfolio:children', 'Portfolio:items')
        super(DM1, self).__init__()
        
    def addDep(self, input, output):
        if not input.key.tweakable:
            if not input.inputs:
                return
        output.inputs.add(input)
        input.outputs.add(output)


    def calculated(self, node):
        if not node.isSimplified:
            return
        for input in node.inputs:
            if input not in node.ctx.tweaks:
                addFootnote(text='context simplification failure', info='%s used %s' % (str(node.key), str(input.key)))
                  
    def getCtx(self, clock):
        ts = clock.cutoffs()
        cKey = (clock, ts) # really want value-based comparison of ts, not object equality
        if cKey not in self.contexts:
            self.contexts[cKey] = Context({clock.cutoffs: ts})
        return self.contexts[cKey]
            
    def getNode(self, ctx, key):
        n = ctx._get(key)
        if n:
            return n
        if key.fullName() in self.simpleClockMethods:
            obj = key.object()
            ctx1 = self.getCtx(obj.clock())
            ret = ctx1.get(key)
            ret.isSimplified = True
        else:
            ret = super(DM1, self).getNode(ctx, key)
            ret.isSimplified = False
        return ret
    
setDependencyManager(DM1())

db4 = db.copy()
valuable = db4.get(b4.meta.path())

clock = valuable.getObj(_tr.RootClock, 'Main')
with Context({clock.cutoffs: ts7}):
    print valuable.NPV()
    node = getNode(valuable.NPV)
    node.printInputGraph()

40.85
<Node: /Global/TradingBook/Rates0.(TradingContainer:NPV)() in Root:4512165784 @4700718608 T=False>, nIn=3
   <Node: /Global/Clock/Trading.(Clock:cutoffs)() in Root:4512165784 @4700717648 T=True>, nIn=2
      <Node: /Global/Clock/Trading.(Clock:parent)() in Root:4512165784 @4700718224 T=False>, nIn=1
         <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4512165784 @4700718480 T=True>, nIn=0
      <Node: /Global/Clock/Workflow.(Clock:cutoffs)() in Root:4512165784 @4738884880 T=True>, nIn=2
         <Node: /Global/Clock/Workflow.(Clock:parent)() in Root:4512165784 @4738884752 T=False>, nIn=1
            <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4512165784 @4700718480 T=True>, nIn=0
         <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4512165784 @4700718480 T=True>, nIn=0
   <Node: /Global/Equity/71475ae5-2823-4c52-b08c-7ef83264d743.(Equity:NPV)() in Root:4512165784 @4607265808 T=False>, nIn=1
      <Node: /Global/MarketInterface/GOOG.

In [7]:
db4 = db.copy()
valuable = db4.get(pAll.meta.path())

r = PnLExplainReport(valuable=valuable, ts1=eod, ts2=ts7)
with ProfileMonitor():
    with SummaryMonitor():
        r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|0.00
|activity: MarketData|5240.00
|activity: Portfolio|1016.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. context simplification failure
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta10>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta11>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta12>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta13>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta14>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta15>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta16>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta17>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta18>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta19>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:children) ()>
1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# Compute activity summary (11.63 seconds of wall clock time)

|key|value|
|-|-|
|Context|19
|Db.Get|2176
|GetValue|28897
|GetValue/Calc|1031


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|11,629,583|18|11,629,583|GetValue
|PnLExplainReport:data|1|11,629,565|459|11,629,565|GetValue/Calc
|TradingContainer:NPV|11|11,620,229|139|1,056,384|GetValue
|TradingContainer:NPV|11|11,620,090|87|1,056,371|GetValue/Calc
|PnLExplainReport:cutoffs|1|11,526,910|15|11,526,910|GetValue
|PnLExplainReport:cutoffs|1|11,526,894|149|11,526,894|GetValue/Calc
|Root:Clocks|47|11,526,647|28,626|245,247|Context
|Portfolio:items|41|11,476,281|6,725|279,909|GetValue
|Portfolio:items|33|11,475,765|261|347,750|GetValue/Calc
|Root:Clocks:4512165784|306|11,468,271|137,800|37,478|Context
|Workbook:items|301|10,822,990|2,722|35,956|GetValue
|Workbook:items|200|10,820,268|1,785|54,101|GetValue/Calc
|Root:Clocks:4512165784:4512165784|400|10,818,483|1,648,773|27,046|Context
|TradeOpenEvent|1,014|4,778,193|4,778,193|4,712|Db.Get
|TradeOpenEvent:ticket|4,052|4,224,512|34,212|1,042|GetValue
|TradingTicket|1,014|4,190,299|4,190,299|4,132|Db.Get
|Portfolio:children|66|582,162|670|8,820|GetValue
|Portfolio:children|33|581,491|321|17,620|GetValue/Calc
|PortfolioUpdateEvent:children|33|396,232|1,699|12,007|GetValue
|TradingBook|102|362,808|362,808|3,556|Db.Get
|Equity:NPV|15|121,708|185|8,113|GetValue
|Equity:NPV|15|121,523|111|8,101|GetValue/Calc
|MarketInterface:spot|15|112,410|149|7,494|GetValue
|MarketInterface:spot|15|112,261|107|7,484|GetValue/Calc
|ExternalRefData:state|15|103,334|162|6,888|GetValue
|ExternalRefData:state|15|103,172|120|6,878|GetValue/Calc
|Clock:cutoffs|870|84,390|3,633|97|GetValue
|Clock:cutoffs|16|80,822|122|5,051|GetValue/Calc
|Clock:parent|16|79,987|167|4,999|GetValue
|Clock:parent|16|79,820|133|4,988|GetValue/Calc
|Root:Amend Portfolio|11|60,895|537|5,535|Context
|Root:Amend Portfolio:4512165784|144|59,944|53,559|416|Context
|RefData:state|15|56,994|90|3,799|GetValue
|RefData:state|4|56,904|33|14,226|GetValue/Calc
|PortfolioUpdateEvent|12|50,133|50,133|4,177|Db.Get
|TradingPortfolio|10|39,001|39,001|3,900|Db.Get
|RefDataUpdateEvent|9|38,367|38,367|4,263|Db.Get
|_WorkItemEvent:book2|4,052|29,031|21,753|7|GetValue
|_WorkItemEvent:item|2,026|26,217|19,204|12|GetValue
|TradingBook:clock|701|23,842|7,455|34|GetValue
|_WorkItemEvent:book1|4,052|23,510|23,510|5|GetValue
|TradeOpenEvent:quantity|4,052|20,927|20,927|5|GetValue
|TradeOpenEvent:premium|2,026|17,796|14,537|8|GetValue
|Clock|5|17,202|17,202|3,440|Db.Get
|TradingBook:clock|500|16,386|4,787|32|GetValue/Calc
|TradeOpenEvent:action|2,026|16,260|16,260|8|GetValue
|Root:End|27|13,145|12,360|486|Context
|Event:amends|2,083|13,064|13,064|6|GetValue
|Root:Start|20|12,652|12,195|632|Context
|TradeOpenEvent:unitPrice|2,026|12,579|12,579|6|GetValue
|Equity:refdata|15|8,247|172|549|GetValue
|MarketInterface:source|15|8,139|162|542|GetValue
|Equity:refdata|15|8,074|144|538|GetValue/Calc
|MarketInterface:source|15|7,977|110|531|GetValue/Calc
|ClockEvent|2|7,796|7,796|3,898|Db.Get
|MarketInterface|2|7,311|7,311|3,655|Db.Get
|Equity|2|7,013|7,013|3,506|Db.Get
|ClockEvent:parent|6|6,936|70|1,156|GetValue
|MarketDataSource|2|6,840|6,840|3,420|Db.Get
|RootClock|1|6,100|6,100|6,100|Db.Get
|Portfolio:clock|140|5,625|1,169|40|GetValue
|Root:Amend MarketData|11|5,150|545|468|Context
|Portfolio:clock|44|4,455|388|101|GetValue/Calc
|Root:Amend MarketData:4512165784|2|4,395|4,321|2,197|Context
|MarketDataSource:clock|23|3,796|262|165|GetValue
|MarketDataSource:clock|19|3,534|182|186|GetValue/Calc
|ForwardCashflow|1|3,258|3,258|3,258|Db.Get
|Portfolio:books|63|2,375|607|37|GetValue
|Portfolio:books|33|1,985|296|60|GetValue/Calc
|Root:Activity Trading|18|1,271|885|70|Context
|Root:Activity Portfolio|18|1,034|726|57|Context
|Entity:clock|32|936|414|29|GetValue
|Root:Start breaks|11|725|512|65|Context
|Root:Amend Trading|11|612|429|55|Context
|Root:Activity MarketData|11|584|396|53|Context
|Entity:clock|16|521|263|32|GetValue/Calc
|MarketInterface:sourceName|15|514|145|34|GetValue
|MarketInterface:sourceName|15|368|110|24|GetValue/Calc
|ForwardCashflow:NPV|11|366|163|33|GetValue
|RootClock:cutoffs|43|243|169|5|GetValue
|ForwardCashflow:NPV|11|203|89|18|GetValue/Calc
|Equity:assetName|15|101|101|6|GetValue
|RefDataUpdateEvent:data|15|81|81|5|GetValue
|RootClock:cutoffs|1|74|40|74|GetValue/Calc
|RootClock:cosmicAll|1|20|10|20|GetValue
|CosmicAll:dbState|1|14|9|14|GetValue
|PnLExplainReport:valuable|2|13|13|6|GetValue
|PnLExplainReport:ts2|2|11|11|5|GetValue
|PnLExplainReport:ts1|2|10|10|5|GetValue
|RootClock:cosmicAll|1|9|9|9|GetValue/Calc
|CosmicAll:dbState|1|5|5|5|GetValue/Calc